#### Task 1

In [1]:
#On March 11, 2020, the World Health Organization (WHO) declared COVID-19, the disease caused by the SARS-CoV-2, a pandemic. Two days later, the Trump Administration declares a nationwide emergency and issues an additional travel ban on non-U.S. citizens traveling from 26 European countries due to COVID-19.

#The two provided csv files described the human mobility pattern of MA residents right before and after the outbreak of the COVID-19 pandemic in the US. Read them into pandas and calculate at least three metrics that we can use to understand the change in people’s mobility patterns due to COVID.

import os
import pandas as pd

os.chdir('/Users/[editted]/Desktop/compsoc/data/MA_cbg_human_mobility_2020')
before_covid = pd.read_csv('MA_cbg2cbg_2020_02.csv', low_memory=False)
#before_covid #(939439*5)
after_covid = pd.read_csv('MA_cbg2cbg_2020_04.csv', low_memory=False)
#after_covid #(384826*5)

In [2]:
# 1 Number of neighborhoods visited
before_covid_num = before_covid.groupby('poi_cbg').agg({'visitor_census_tract': 'count'}).reset_index()
print(before_covid_num['visitor_census_tract'].describe(), "\n")
after_covid_num = after_covid.groupby('poi_cbg').agg({'visitor_census_tract': 'count'}).reset_index()
print(after_covid_num['visitor_census_tract'].describe())

count     4984.000000
mean       188.490971
std        282.346169
min          1.000000
25%         71.000000
50%        121.000000
75%        214.000000
max      10682.000000
Name: visitor_census_tract, dtype: float64 

count    4984.000000
mean       77.212279
std        81.046065
min         2.000000
25%        34.000000
50%        55.000000
75%        89.000000
max      1870.000000
Name: visitor_census_tract, dtype: float64


**Task 1 #1 Conclusion**
In this approach, I'm calculating the average number of neighborhoods visited for each census block group (cbg) before and after the COVID pandemic. In February, which is before COVID, the average number of neighborhoods visited is about 188. However, in April, the average number of neighborhoods visited decreased to about 77. After the outbreak, the frequency of people visiting neighborhoods dropped significantly.

In [3]:
# 2 Number of visitors
before_covid_visitors = before_covid.groupby('poi_cbg').agg({'visitor_count': 'sum'}).reset_index()
before_covid_visitors = before_covid_visitors.sort_values(by=['poi_cbg'], ascending=True)
#before_covid_visitors #(4984*2)
after_covid_visitors = after_covid.groupby('poi_cbg').agg({'visitor_count': 'sum'}).reset_index()
#after_covid_visitors #(4984*2)
compare_visitors = before_covid_visitors.merge(after_covid_visitors, left_on='poi_cbg', right_on='poi_cbg')
compare_visitors['difference_#_of_visitors'] = compare_visitors['visitor_count_x'] - compare_visitors['visitor_count_y']
print((compare_visitors['difference_#_of_visitors']<0).sum())
print((compare_visitors['difference_#_of_visitors']>0).sum(), "\n")
print(compare_visitors['difference_#_of_visitors'].describe())

92
4890 

count     4984.000000
mean       745.934992
std       1549.116109
min      -7080.000000
25%        177.000000
50%        376.000000
75%        814.000000
max      65027.000000
Name: difference_#_of_visitors, dtype: float64


**Task 1 #2 Conclusion**
In this approach, I compared the change in number of visitors from cbgs. The result shows that although there are certain numbers of cbgs that increase in visitors, most of the cbgs decrease in the number of visitors with an average decrease of 745 visitors per cbg. This shows that after the pandemic, there are fewer visitors traveling around.

In [4]:
# 3 Residents flow
before_covid_residents = before_covid.drop(['year_month','visitor_census_tract','visitor_count'], axis=1)
before_covid_residents = before_covid_residents.drop_duplicates(subset=['poi_cbg'])
before_covid_residents = before_covid_residents.sort_values(by=['poi_cbg'], ascending=True)
#before_covid_residents #(4984*2)
after_covid_residents = after_covid.drop(['year_month','visitor_census_tract','visitor_count'], axis=1)
after_covid_residents = after_covid_residents.drop_duplicates(subset=['poi_cbg'])
after_covid_residents = after_covid_residents.sort_values(by=['poi_cbg'], ascending=True)
#after_covid_residents #(4984*2)
compare_residents = before_covid_residents.merge(after_covid_residents, left_on='poi_cbg', right_on='poi_cbg')
compare_residents['raw_device_counts_difference'] = compare_residents['raw_device_counts_x'] - compare_residents['raw_device_counts_y']
print((compare_residents['raw_device_counts_difference']<0).sum())
print((compare_residents['raw_device_counts_difference']>0).sum(), "\n")
print(compare_residents['raw_device_counts_difference'].describe())

59
4923 

count     4984.000000
mean       969.745385
std       2036.366301
min     -11443.000000
25%        216.000000
50%        452.000000
75%        986.250000
max      76067.000000
Name: raw_device_counts_difference, dtype: float64


**Task 1 #3 Conclusion**
In this approach, I compared the change in number of residents in cbgs after the pandemic. The result turns out similar as the previous approach that although there are a certain number of cbgs that have increasing number of residents after the pandemic, most of cbgs have significant decrease in their number of residents, with an average at around 969 residents decrease per cbg. It implies that people tend to flee from MA after the outbreak (maybe because they're afraid that MA will be the next NY, where a lot of dead cases were reported).

#### Task 2

In [5]:
#Use these two data sources to create two human mobility networks as directed graphs for MA with

#network node: individual census block group (neighborhood) in MA
#network edge: residents from one neighborhood visiting another with the direction of the edge implying the direction of the visit
#weight of network tie: percentage of residents from one neighborhood (origin) visiting another (destination)

#Calculate at least three network-based metrics that we can use to understand the change in people’s mobility due to COVID.

import os
import pandas as pd
import networkx as nx

os.chdir('/Users/[editted]/Desktop/compsoc/data/MA_cbg_human_mobility_2020')
before_covid = pd.read_csv('MA_cbg2cbg_2020_02.csv', low_memory=False)
after_covid = pd.read_csv('MA_cbg2cbg_2020_04.csv', low_memory=False)

# Human Mobility Network
G_before = nx.Graph()
G_after = nx.Graph()
for i in range(len(before_covid)):
    origin_cbg = before_covid.loc[i, 'poi_cbg']
    destination_cbg = before_covid.loc[i, 'visitor_census_tract']
    if str(origin_cbg)[:2]=='25' and str(destination_cbg)[:2]=='25':
        total_residents_in_origin = before_covid.loc[i, 'raw_device_counts']
        num_visitor = before_covid.loc[i, 'visitor_count']
        percent_visitor = num_visitor / total_residents_in_origin
        G_before.add_edge(origin_cbg, destination_cbg, weight=percent_visitor)
print(nx.info(G_before))
for i in range(len(after_covid)):
    origin_cbg = after_covid.loc[i, 'poi_cbg']
    destination_cbg = after_covid.loc[i, 'visitor_census_tract']
    if str(origin_cbg)[:2]=='25' and str(destination_cbg)[:2]=='25':
        total_residents_in_origin = after_covid.loc[i, 'raw_device_counts']
        num_visitor = after_covid.loc[i, 'visitor_count']
        percent_visitor = num_visitor / total_residents_in_origin
        G_after.add_edge(origin_cbg, destination_cbg, weight=percent_visitor)
print(nx.info(G_after))

/var/folders/ml/2z7dml2d3vn2rqmr8h2sw_n40000gn/T/ipykernel_58485/3603284743.py:28: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(G_before))


Graph with 4984 nodes and 635681 edges
Graph with 4985 nodes and 291120 edges


/var/folders/ml/2z7dml2d3vn2rqmr8h2sw_n40000gn/T/ipykernel_58485/3603284743.py:37: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(G_after))


In [6]:
# Degree difference (difference between # of edges adjacent to the node)
node_list_Feb = list(G_before.nodes())
degree_before = pd.DataFrame({'poi_cbg': node_list_Feb})
#degree_before
node_list_Apr = list(G_after.nodes())
degree_after = pd.DataFrame({'poi_cbg': node_list_Apr})
#degree_after
node_degree_Feb = []
for node in G_before.nodes():
    node_deg = G_before.degree(node)
    node_degree_Feb.append(node_deg)
degree_before = degree_before.join(pd.DataFrame({'degree_before': node_degree_Feb}))
node_degree_Apr = []
for node in G_after.nodes():
    node_deg = G_after.degree(node)
    node_degree_Apr.append(node_deg)
degree_after = degree_after.join(pd.DataFrame({'degree_after': node_degree_Apr}))
degree_all = degree_before.merge(degree_after, left_on='poi_cbg', right_on='poi_cbg')
degree_all['degree_diff'] = degree_all['degree_before'] - degree_all['degree_after']
print(degree_all['degree_diff'].describe())

count    4984.000000
mean      138.267255
std       136.071291
min       -93.000000
25%        71.000000
50%       109.000000
75%       166.000000
max      3143.000000
Name: degree_diff, dtype: float64


**Task 2 #1 Conclusion**
In this approach, I compared the difference between the node degrees (unweighted), which means the difference between the number of edges connected to a certain node before and after the pandemic announced. The finding is that on average, the degree decreased by 138, althogh a small amount of cbgs having increasing degrees. Less connection between nodes means that people are travelling less between neighborhoods.

In [7]:
# Closeness Centrality
import numpy as np

cc_before = nx.closeness_centrality(G_before)
average_cc_before = np.array(list(cc_before.values())).mean()
print(average_cc_before)
cc_after = nx.closeness_centrality(G_after)
average_cc_after = np.array(list(cc_after.values())).mean()
print(average_cc_after)

0.49957041134479185
0.4225415660281424


**Task 2 #2 Conclusion**
In this approach, I compared the average closeness centrality of the two networks. The value represents the average distance from a node to other nodes for all nodes in the two networks. Nodes with higher closeness centrality have the shorter distances to all nodes, which, in this context, means that the neighborhoods were more closely related to each other before the announcement of the pandemic than after the announcement of the pandemic (0.4996>0.4225).

In [8]:
# Clustering Coefficient
ccoeff_before = nx.average_clustering(G_before)
print(ccoeff_before)
ccoeff_after = nx.average_clustering(G_after)
print(ccoeff_after)

0.3358877036019045
0.2799481682222787


**Task 2 #3 Conclusion**
In this approach, I compared the average clustering coefficients of the two networks. Clustering coefficient measures the average probability that two neighbors of a node are neighbors, which means that a higher clustering coefficient means that the neighbors are more closely related. In this case, we can see that the average clustering coefficient in February is higher than that in April, implying that the neighborhoods are more closely related (perhaps more visitors travel in between) before the announcement of the pandemic.